In [1]:
import keras
import tensorflow as tf
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
IMAGE_WIDTH = 300
IMAGE_HEIGHT = 300
IMAGE_CHANNELS = 1
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
NUM_CLASSES = 4
IMG_DIR = 'F:\Self Study\Datasets\mask_dataset_images\imagens'
BATCH_SIZE = 32

In [2]:
base_model = keras.applications.MobileNetV2(
    input_shape=(IMAGE_WIDTH,IMAGE_HEIGHT,IMAGE_CHANNELS),
    weights=None,
    include_top=False,
)

In [3]:
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(256,activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)

predictions = keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)
model = keras.Model(inputs=base_model.input, outputs=predictions)

#opt = keras.optimizers.Adam(lr=0.000125)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [4]:
callbacks_list = [
    ModelCheckpoint(
    'Models/mask_detection/mask_weights.h5', monitor='val_accuracy',verbose=1, save_best_only=True, mode='max'),
    EarlyStopping(monitor='val_accuracy', patience=5),
    ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
]

In [6]:
def add_noise(img):
    '''Add random noise to an image'''
    VARIABILITY = 8
    deviation = VARIABILITY*np.random.random()
    noise = np.random.normal(0, deviation, img.shape)
    img += noise
    np.clip(img, 0., 255.)
    return img

train_datagen = ImageDataGenerator(
    brightness_range=[0.4, 1.6],
    rescale=1. / 255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    preprocessing_function=add_noise,
    validation_split=0.2
)
train_generator = train_datagen.flow_from_directory(
    IMG_DIR,
    target_size=IMAGE_SIZE,
    color_mode = 'grayscale',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False,
    subset='training'
)

validation_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)
validation_generator = validation_datagen.flow_from_directory(
    IMG_DIR,
    target_size=IMAGE_SIZE,
    color_mode = 'grayscale',
    class_mode='categorical',
    shuffle=False,
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 139116 images belonging to 4 classes.
Found 34777 images belonging to 4 classes.


In [8]:
history = model.fit_generator(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.n//BATCH_SIZE,
    steps_per_epoch=train_generator.n//BATCH_SIZE,
    callbacks=callbacks_list
)

C:\Users\liors\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
   5/4347 [..............................] - ETA: 6:22:16 - loss: 3.4980 - accuracy: 0.0608

KeyboardInterrupt: 

### After training on google-colab, load and evaluate the model on the validation set:

In [22]:
mask_model = tf.keras.models.load_model('Models/mask_detection/model.h5')
mask_model.load_weights('Models/mask_detection/mask_weights.h5')

In [23]:
results = mask_model.evaluate(validation_generator)

1087/1087 [==============================] - 702s 645ms/step - loss: 0.0014 - accuracy: 0.9996


## Accuracy of 0.9996 with a 0.0014 loss, perfect results.